In [1]:
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")
using PyCall
using Distributions
unshift!(PyVector(pyimport("sys")["path"]), "")
unshift!(PyVector(pyimport("sys")["path"]), "..")
unshift!(PyVector(pyimport("sys")["path"]), "../Ad05RTheta1RSteer05Abs0FR200")
unshift!(PyVector(pyimport("sys")["path"]), "../CommonFiles")
@pyimport python2juliaReuse as p2j

In [2]:
# policies = ["Baseline/Rewards/b_15e3_3","Retrain/Rewards/rarl","Retrain/Rewards/rarlP10",
#             "FSP/Rewards/nBnoTR","FSP/Rewards/nBnoTRP10"]

In [3]:
pc = p2j.PolicyContainer()
path = "Data/Sep1/Ad05RTheta1RSteer05Abs0FR200/"
# policies = readdir(string(path,"Policy"))
# pnames = []
# for i = 1:length(policies)
#     if policies[i] != ".DS_Store"
#         push!(pnames,policies[i][1:length(policies[i])-4])
#     end
# end
# println(pnames)
pnames = ["b_15e3","rarl_5e3","fsp_5e3"]
include("../Ad05RTheta1RSteer05Abs0FR200/preloadRARL.jl")


2018-09-06 23:31:59.678184: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-09-06 23:31:59.678200: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2018-09-06 23:31:59.678204: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
2018-09-06 23:31:59.678206: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.
06-Sep 23:32:00:WARNING:root:replacing docs for 'close :: Tuple{ZMQTransport}' in module 'Main'.
06-Sep 23:32:00

run_env_server_rarl (generic function with 1 method)

In [4]:
mutable struct Counter
    xs::Array{Float64}
    obs::Array{Array{Float64}}
    rewards::Array{Float64}
    accs::Array{Float64}
    steers::Array{Float64}
    daccs::Array{Float64}
    dsteers::Array{Float64}
    done::Bool
end
function Counter()
    return Counter([],[],[],[],[],[],[],false)
end

Counter

In [5]:
function handle_action_pareto!(pc,scene,models,roadway,counter,rarl,pareto,reverse)
    append!(counter.xs,scene[1].state.posG.x)
    
    state = get_observation(scene,models,roadway)
    append!(counter.obs,[state])
    if rarl
#         action1_arr = pc[:getAction_just](state)
        dist = pc[:getActionDistribution_just](state)
        action1_arr = dist["mean"]
    else
        action1_arr = pc[:getAction_just_baseline](state)
    end
    action1_arr = tanh.(action1_arr)

    dacc = rand(pareto)-2.0
    dsteer = rand(pareto)-2.0
    if reverse
        dacc = -dacc
        dsteer = -dsteer
    end
    action = Egoaction(action1_arr[1],action1_arr[2],dacc,dsteer)
    
    append!(counter.accs,action1_arr[1])
    append!(counter.steers,action1_arr[2])
    append!(counter.daccs,clamp(dacc,-1.0,1.0))
    append!(counter.dsteers,clamp(dsteer,-1.0,1.0))
    
    done, bump = simulate_action!(action,scene,models,roadway)
    reward = reward_fn(action,done,scene,models,roadway)

    append!(counter.rewards,reward)
    if done
        counter.done = true
    end
    
end

handle_action_pareto! (generic function with 1 method)

In [7]:
alphas = [1,2,3,4,5]
log_path = string(path,"SimuTraj/Pareto10r/")
scene, models, roadway = initialize_env()
for alpha in alphas
    println("alpha :",alpha)
    pareto=Pareto(10,1)
    for pname in pnames
        println(pname)
        policy_path = string(path,"Policy/",pname,".pkl")
        pc[:reset_policy](policy_path=policy_path)

        counter = Counter()
        rand_ego!(scene,models,roadway)
        pc[:resetPolicy](1)
        while !counter.done
            handle_action_pareto!(pc,scene,models,roadway,counter,true,pareto,true)
        end

        writedlm(string(log_path,pname,"_xs",alpha,".txt"),counter.xs)
        writedlm(string(log_path,pname,"_obs",alpha,".txt"),counter.obs)
        writedlm(string(log_path,pname,"_rewards",alpha,".txt"),counter.rewards)
        writedlm(string(log_path,pname,"_accs",alpha,".txt"),counter.accs)
        writedlm(string(log_path,pname,"_steers",alpha,".txt"),counter.steers)
        writedlm(string(log_path,pname,"_daccs",alpha,".txt"),counter.daccs)
        writedlm(string(log_path,pname,"_dtseers",alpha,".txt"),counter.dsteers)
    end
end

alpha :1
b_15e3
rarl_5e3
fsp_5e3
alpha :2
b_15e3
rarl_5e3
fsp_5e3
alpha :3
b_15e3
rarl_5e3
fsp_5e3
alpha :4
b_15e3
rarl_5e3
fsp_5e3
alpha :5
b_15e3
rarl_5e3
fsp_5e3


In [ ]:
# mutable struct Counter
#     xs::Array{Float64}
#     obs::Array{Float64}
#     rewards::Array{Float64}
#     accs::Array{Float64}
#     steers::Array{Float64}
#     daccs::Array{Float64}
#     dsteers::Array{Float64}
#     done::Bool
# end
# function Counter()
#     return Counter([],[],[],[],[],[],[],false)
# end